In [2]:
import pandas as pd

data = pd.read_excel('./data/total_accident_info.xlsx')

In [3]:
data['시군구'] = data['시군구'].str.replace('서울특별시','')

In [4]:
data

,시군구,사고내용,사망자수,중상자수,경상자수,부상신고자수,부상자수,사고유형,법규위반,도로형태
0,중구 충무로5가,경상사고,0,0,1,0,1,차대차 - 기타,안전운전불이행,단일로 - 기타
1,송파구 신천동,중상사고,0,2,0,0,2,차대사람 - 횡단중,보행자보호의무위반,단일로 - 기타
2,금천구 독산동,부상신고사고,0,0,0,1,1,차량단독 - 기타,안전운전불이행,단일로 - 기타
3,강동구 천호동,경상사고,0,0,2,0,2,차대차 - 측면충돌,안전운전불이행,단일로 - 기타
4,강남구 역삼동,중상사고,0,1,0,0,1,차대사람 - 기타,안전운전불이행,교차로 - 교차로안
...,...,...,...,...,...,...,...,...,...,...
33693,광진구 군자동,경상사고,0,0,1,0,1,차대차 - 기타,안전거리미확보,단일로 - 기타
33694,마포구 염리동,경상사고,0,0,1,0,1,차대차 - 추돌,안전운전불이행,단일로 - 기타
33695,중랑구 면목동,부상신고사고,0,0,0,1,1,차대차 - 측면충돌,차로위반,단일로 - 기타
33696,구로구 구로동,경상사고,0,0,1,0,1,차대차 - 측면충돌,안전운전불이행,교차로 - 교차로부근


In [5]:
data['자치구'] = data['시군구'].apply(lambda x: x.split()[0])

In [6]:
data = data.rename(columns={'시군구' : '동', '부상지수' : '사상자수'})

In [7]:
data['동'] = data['동'].apply(lambda x: x.split()[-1])

In [8]:
df = pd.DataFrame({'자치구' : data['자치구'], '동' : data['동'], '부상자수' : data['부상자수']})

In [9]:
df

,자치구,동,부상자수
0,중구,충무로5가,1
1,송파구,신천동,2
2,금천구,독산동,1
3,강동구,천호동,2
4,강남구,역삼동,1
...,...,...,...
33693,광진구,군자동,1
33694,마포구,염리동,1
33695,중랑구,면목동,1
33696,구로구,구로동,1


In [10]:
df_sorted = df.sort_values(by='자치구')

In [11]:
df_sorted = df_sorted.reset_index(drop=True)
df_sorted

,자치구,동,부상자수
0,강남구,역삼동,3
1,강남구,개포동,1
2,강남구,신사동,1
3,강남구,개포동,1
4,강남구,청담동,3
...,...,...,...
33693,중랑구,면목동,1
33694,중랑구,신내동,1
33695,중랑구,중화동,1
33696,중랑구,망우동,1


In [12]:
separate_dfs = {name: group for name, group in df_sorted.groupby('자치구')}

In [13]:
separate_dfs['강남구']

,자치구,동,부상자수
0,강남구,역삼동,3
1,강남구,개포동,1
2,강남구,신사동,1
3,강남구,개포동,1
4,강남구,청담동,3
...,...,...,...
3704,강남구,논현동,2
3705,강남구,논현동,2
3706,강남구,대치동,3
3707,강남구,신사동,2


In [14]:
re_df = separate_dfs['강남구'].groupby(['자치구', '동']).agg({'부상자수' : 'sum'}).reset_index()

re_df_gr = re_df.groupby('자치구').agg({'부상자수' : 'sum'}).reset_index()
re_df_gr['동'] = '합계'

fi_df = pd.concat([re_df, re_df_gr], ignore_index=True)

print(fi_df)

    자치구     동  부상자수
0   강남구   개포동   314
1   강남구   논현동   722
2   강남구   대치동   653
3   강남구   도곡동   334
4   강남구   삼성동   513
5   강남구   세곡동    64
6   강남구   수서동   157
7   강남구   신사동   454
8   강남구  압구정동   256
9   강남구   역삼동  1015
10  강남구   율현동    27
11  강남구   일원동   105
12  강남구   자곡동    60
13  강남구   청담동   394
14  강남구    합계  5068


In [15]:
# 모든 자치구에 대해 부상자수 합계를 계산하여 새로운 데이터프레임 생성
result_dfs = {}
for name, separate_df in separate_dfs.items():
    re_df = separate_df.groupby(['자치구', '동']).agg({'부상자수': 'sum'}).reset_index()
    re_df_gr = re_df.groupby('자치구').agg({'부상자수': 'sum'}).reset_index()
    re_df_gr['동'] = '합계'
    result_dfs[name] = pd.concat([re_df, re_df_gr], ignore_index=True)

# 결과 출력
for name, result_df in result_dfs.items():
    print(f"자치구: {name}")
    print(result_df)
    print("--------------------")

자치구: 강남구
    자치구     동  부상자수
0   강남구   개포동   314
1   강남구   논현동   722
2   강남구   대치동   653
3   강남구   도곡동   334
4   강남구   삼성동   513
5   강남구   세곡동    64
6   강남구   수서동   157
7   강남구   신사동   454
8   강남구  압구정동   256
9   강남구   역삼동  1015
10  강남구   율현동    27
11  강남구   일원동   105
12  강남구   자곡동    60
13  강남구   청담동   394
14  강남구    합계  5068
--------------------
자치구: 강동구
   자치구    동  부상자수
0  강동구  강일동   152
1  강동구  고덕동   150
2  강동구   길동   182
3  강동구  둔촌동   130
4  강동구  명일동   132
5  강동구  상일동   131
6  강동구  성내동   212
7  강동구  암사동   172
8  강동구  천호동   299
9  강동구   합계  1560
--------------------
자치구: 강북구
   자치구    동  부상자수
0  강북구  미아동   555
1  강북구   번동   255
2  강북구  수유동   433
3  강북구  우이동    29
4  강북구   합계  1272
--------------------
자치구: 강서구
    자치구     동  부상자수
0   강서구   가양동   168
1   강서구   개화동    83
2   강서구   공항동   105
3   강서구   과해동     6
4   강서구  내발산동   149
5   강서구   등촌동   252
6   강서구   마곡동   258
7   강서구   방화동   217
8   강서구   염창동   117
9   강서구   오곡동     5
10  강서구   오쇠동     3
11  강서구  외발산동    66
12  강서구   화곡동  

In [16]:
# 2022년 총 교통사고 수
# 자치구 이름을 키값으로 '동'별 부상자수를 구했습니다.
result_dfs.keys()

dict_keys(['강남구', '강동구', '강북구', '강서구', '관악구', '광진구', '구로구', '금천구', '노원구', '도봉구', '동대문구', '동작구', '마포구', '서대문구', '서초구', '성동구', '성북구', '송파구', '양천구', '영등포구', '용산구', '은평구', '종로구', '중구', '중랑구'])

In [17]:
result_dfs['강남구']

,자치구,동,부상자수
0,강남구,개포동,314
1,강남구,논현동,722
2,강남구,대치동,653
3,강남구,도곡동,334
4,강남구,삼성동,513
5,강남구,세곡동,64
6,강남구,수서동,157
7,강남구,신사동,454
8,강남구,압구정동,256
9,강남구,역삼동,1015
